# Neural Language Model
- 목적 : 
- Output : 뒤의 단어 예측
- Model : FCN
- Loss : Cross Entropy

##### 0) 라이브러리 불러오기

In [7]:
# (1) 딥러닝 라이브러리
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

##### 1) 데이터 불러오기

(1) 변수 초기화

In [8]:
NGRAM_SIZE = 2
EMBEDDING_DIM = 10

(2) 데이터 불러오기 (셰익스피어 소네트2)

In [9]:
sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-NGRAM_SIZE, ..., word_i-1 ], target word)

In [10]:
sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

##### 2) N-gram

In [11]:
ngrams = [([sentence[i - j - 1] for j in range(NGRAM_SIZE)], sentence[i]) for i in range(NGRAM_SIZE, len(sentence))] # !!!

print(ngrams[:3])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


##### 3) 단어 사전 만들기

In [12]:
vocab = set(sentence)
word_to_idx = {word: i for i, word in enumerate(vocab)}

In [ ]:
len(vocab)

## Ngram pytorch module class 설정

##### 4) 모델 정의

In [35]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, EMBEDDING_DIM, NGRAM_SIZE):
        super(NGramLanguageModeler, self).__init__() # !
        self.embeddings = nn.Embedding(vocab_size, embedding_dim=EMBEDDING_DIM) # !!!
        self.linear1 = nn.Linear(NGRAM_SIZE * EMBEDDING_DIM, 128) # !!!
        self.linear2 = nn.Linear(128, vocab_size) # !!
    def forward(self, inputs):
        # print(f'inputs : {inputs}') # tensor([51, 64])
        embeds = self.embeddings(inputs).view((1, -1)) # !!!
        # print(f'embeds : {embeds}')
        """
        tensor([[-0.7379, -0.2725, -1.2734, -0.5576,  0.0750,  0.4588, -0.9168, -0.1681,
         -1.8120, -0.1912, -0.4644,  0.0171, -0.0437,  1.1693,  1.1770, -0.2649,
          1.0498,  0.4795, -0.7503,  1.0908]], grad_fn=<ViewBackward0>)
        """
        output = F.relu(self.linear1(embeds)) # !!!
        output = self.linear2(output) # !!!
        log_probs = F.log_softmax(output, dim=1) # 열방향으로 Log Softmax # !!!
        # print(f'log_probs : {log_probs}')
        """
        tensor([[-4.7832, -4.7157, -4.7879, -4.5430, -4.4339, -4.5025, -4.2623, -4.0010,
         -4.7352, -4.6143, -4.3309, -4.6151, -4.3560, -4.4804, -4.7485, -4.5814,
         -4.6898, -4.2726, -4.9978, -4.6067, -4.1026, -4.1331, -4.5865, -4.7218,
         -4.7840, -4.6431, -4.7147, -4.6294, -4.7187, -4.2708, -4.6978, -4.4084,
         -4.7089, -4.2476, -4.4089, -4.6950, -4.6561, -4.9805, -4.7100, -4.6702,
         -4.6025, -4.3475, -4.6389, -4.8811, -4.6336, -4.9418, -4.8256, -5.0364,
         -4.3126, -4.4835, -4.3877, -4.8331, -4.5688, -4.8238, -4.7488, -4.7655,
         -4.7157, -4.4630, -4.4461, -4.5065, -4.6963, -5.0180, -4.6433, -4.2379,
         -4.5765, -4.7061, -4.6288, -4.7156, -4.8744, -4.7938, -4.5158, -4.5270,
         -4.6661, -4.4159, -4.7725, -4.5816, -4.4162, -4.6457, -4.4713, -4.6018,
         -4.6241, -4.4262, -4.5049, -4.2798, -4.6101, -4.8002, -4.4954, -4.9527,
         -4.6375, -4.5028, -4.7188, -4.0668, -4.7514, -4.6962, -4.8468, -4.4820,
         -4.5518]], grad_fn=<LogSoftmaxBackward0>)
        """
        return log_probs
    
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, NGRAM_SIZE)

In [36]:
print(model)

NGramLanguageModeler(
  (embeddings): Embedding(97, 10)
  (linear1): Linear(in_features=20, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)


##### 5) Optimzier 정의

In [37]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

##### 6) 모델 학습

In [38]:
losses = []
loss_function = nn.NLLLoss() # !!!

# (1) Epoch 순회
num_epochs = 100
for epoch in range(num_epochs + 1):
    # 1] 변수 초기화
    total_loss = 0
    for context, next_word in ngrams:
        # 2] train 데이터 전처리
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words into integer indices and wrap them in tensors)
        context_idxes = torch.tensor([word_to_idx[c] for c in context], dtype=torch.long) # !!! 원핫 임베딩 형태가 아닌 idx 형태를 넣음

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a new instance, you need to zero out the gradients from the old instance
        model.zero_grad()

        # 3] 모델 예측
        # Step 3. Run the forward pass, getting log probabilities over next words
        log_probs = model(context_idxes)

        # 4] Cost 계산
        # Step 4. Compute your loss function. (Again, Torch wants the target word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_idx[next_word]], dtype=torch.long)) # !!! 원핫 임베딩 형태가 아닌 idx 형태를 넣음
        # print(torch.tensor([word_to_idx[next_word]], dtype=torch.long)) # tensor([61])

        # 5] 모델 역전파
        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)

# 6] 결과 출력
print(f'losses : {losses}')

print(model.embeddings.weight[word_to_idx["beauty"]]) # !!!

tensor([61])
tensor([74])
tensor([62])
tensor([46])
tensor([84])
tensor([66])
tensor([92])
tensor([39])
tensor([30])
tensor([34])
tensor([46])
tensor([33])
tensor([47])
tensor([32])
tensor([93])
tensor([65])
tensor([48])
tensor([73])
tensor([96])
tensor([56])
tensor([9])
tensor([80])
tensor([5])
tensor([42])
tensor([72])
tensor([88])
tensor([35])
tensor([13])
tensor([7])
tensor([31])
tensor([22])
tensor([83])
tensor([50])
tensor([85])
tensor([36])
tensor([46])
tensor([68])
tensor([67])
tensor([54])
tensor([36])
tensor([17])
tensor([63])
tensor([35])
tensor([46])
tensor([58])
tensor([3])
tensor([41])
tensor([19])
tensor([23])
tensor([38])
tensor([20])
tensor([39])
tensor([55])
tensor([60])
tensor([2])
tensor([81])
tensor([40])
tensor([29])
tensor([75])
tensor([28])
tensor([24])
tensor([94])
tensor([14])
tensor([69])
tensor([57])
tensor([0])
tensor([46])
tensor([33])
tensor([95])
tensor([1])
tensor([49])
tensor([52])
tensor([26])
tensor([76])
tensor([25])
tensor([6])
tensor([35])
tensor(